In [1]:
%%loadFromPOM
<repository>
  <id>maven.repo.local</id>
  <url>/home/maho/.m2</url>
</repository>

<properties>

    <pebble.version>3.1.5</pebble.version>
    <springcloud.version>2021.0.3</springcloud.version>
    <springdoc.version>1.6.9</springdoc.version>
    <vertx.version>4.3.1</vertx.version>
</properties>


<dependency>
    <groupId>io.vertx</groupId>
    <artifactId>vertx-core</artifactId>
    <version>${vertx.version}</version>
</dependency>
<dependency>
    <groupId>io.vertx</groupId>
    <artifactId>vertx-web</artifactId>
    <version>${vertx.version}</version>
</dependency>

<dependency>
    <groupId>io.vertx</groupId>
    <artifactId>vertx-web-client</artifactId>
    <version>${vertx.version}</version>
</dependency>



<!-- https://mvnrepository.com/artifact/org.slf4j/slf4j-api -->
<dependency>
    <groupId>org.slf4j</groupId>
    <artifactId>slf4j-api</artifactId>
   <version>2.0.0</version> 
</dependency>



In [2]:


import io.vertx.core.Vertx;
import io.vertx.core.buffer.Buffer;
import io.vertx.core.AbstractVerticle;
import io.vertx.ext.web.client.HttpRequest;
import io.vertx.ext.web.client.WebClient;
import io.vertx.ext.web.client.WebClientOptions;

import java.util.logging.Level;
import java.util.logging.Logger;
import java.util.concurrent.ConcurrentHashMap;
import java.util.concurrent.atomic.AtomicInteger;
import org.slf4j.Logger;
import org.slf4j.LoggerFactory;
// import ch.qos.logback.classic.util.ContextInitializer;
// import java.util.logging.Level;
// import java.util.logging.Logger;
var out = System.out;


In [3]:
// 注意，此下面的slf4j-simple import不能跟slf4j-api放在一起，不然看起来会覆盖掉slf4j-api，其他的一些import似乎也有这个问题

In [4]:
%%loadFromPOM
<dependency>
    <groupId>org.slf4j</groupId>
    <artifactId>slf4j-simple</artifactId>
    <version>2.0.0</version>
    <scope>runtime</scope>
</dependency>


In [8]:
//每个请求的开始时间
ConcurrentHashMap<Integer, Long> begins = new ConcurrentHashMap<>();

//每个请求的结束时间，-1代表请求出错
ConcurrentHashMap<Integer, Long> ends = new ConcurrentHashMap<>();

//默认一秒为一个period，每个period开始，利用vert发出所有请求。
int concurrentPeriodTime = 1000;
//每个period发的次数，即并发数
int concurrentPerPeriod = 500;
AtomicInteger currentRequestId = new AtomicInteger(0);

Logger logger = LoggerFactory.getLogger("jupyter");

In [6]:
public class RedisVerticle extends AbstractVerticle {
    private final Logger logger = LoggerFactory.getLogger(getClass());

    //每个请求的开始时间
    ConcurrentHashMap<Integer, Long> begins = new ConcurrentHashMap<>();

    //每个请求的结束时间，-1代表请求出错
    ConcurrentHashMap<Integer, Long> ends = new ConcurrentHashMap<>();

    //默认一秒为一个period，每个period开始，利用vert发出所有请求。
    int concurrentPeriodTime = 1000;
    //每个period发的次数，即并发数
    int concurrentPerPeriod = 500;
    AtomicInteger currentRequestId = new AtomicInteger(0);

    void logRequestStatus() {
        int tot = 0;
        var list = Collections.list(ends.keys());
        list.sort(Comparator.reverseOrder());

        int cnt = 0;
        for (int i : list) {
            if (ends.get(i) != -1) {
                tot += ends.get(i) - begins.get(i);
            }
            if (++cnt >= 1000) {
                break;
            }
        }
        logger.info("recent {} requests cost average {} ms, {} requests complete", cnt, (double) tot / cnt, list.size());
    }

    void periodConcurrent() {
        WebClient client = WebClient.create(vertx);
        WebClientOptions options = new WebClientOptions()
                .setUserAgent("My-App/1.2.3");
        options.setKeepAlive(false);
        long produceRequestTime = System.currentTimeMillis();
        for (int i = 0; i < concurrentPerPeriod; i++) {
            int finalId = currentRequestId.getAndAdd(1);
            begins.put(finalId, System.currentTimeMillis());
            HttpRequest<Buffer> localhost = client
                    .get(8000, "localhost", "/api/ticks");

            localhost.send()
                    .onSuccess(response -> {
                        long endTime = System.currentTimeMillis();
                        ends.put(finalId, endTime);

                    })
                    .onFailure(err -> {
                        logger.error("Something went wrong " + finalId + " " + err.getMessage());
                        ends.put(finalId, -1L);
                    });
        }
        logger.info("produce request cost {} ms", System.currentTimeMillis() - produceRequestTime);
    }

    @Override
    public void start() throws Exception {
        vertx.setPeriodic(3000, id -> {
            logRequestStatus();
        });

        vertx.setPeriodic(concurrentPeriodTime, id -> {
            periodConcurrent();
        });
    }

    public static void main(String[] args) {
        Vertx vert = Vertx.vertx();
        var push = new RedisVerticle();
        vert.deployVerticle(push);
    }


}

In [ ]:
Vertx vert = Vertx.vertx();
var push = new RedisVerticle();
vert.deployVerticle(push);
Thread.sleep(10000);//这是必须的，不然读不到标准输出。